In [7]:
import numpy as np
import os
import os.path as op
from ipyparallel import Client

In [2]:
engines = Client()

In [3]:
home_dir =  '/home/dnl/habitization/'
subs = home_dir + 'subjects.txt'
subs = list(np.loadtxt(subs,str))
subjects_dir = home_dir + '/data/freesurfer'

In [10]:
def recon(sub):
    cmd = ['recon-all', 
           '-s',sub,
           '-i',op.join(home_dir,'data',sub,'anat','T1_a.nii.gz'),
           '-i',op.join(home_dir,'data',sub,'anat','T1_b.nii.gz'),
           '-sd',op.join(home_dir,'data','freesurfer'),
           '-all']
    cmd = ' '.join(cmd)
    os.system(cmd)
    return sub

recon-all -s HAB25 -i /home/dnl/habitization/data/HAB25/anat/T1_a.nii.gz -i /home/dnl/habitization/data/HAB25/anat/T1_b.nii.gz -sd /home/dnl/habitization/data/freesurfer -all


'HAB25'

In [ ]:
sub_engines = engines[0:11] #less if you want to use fewer
sub_engines.push(dict(home_dir = home_dir))
sub_engines.execute('import os.path as op')
with sub_engines.sync_imports():
    import os
    
output = sub_engines.map_sync(recon,subs)

importing os on engine(s)


In [9]:
##fix directory structure. For some reason ipython insists on making a new directory for freesurfer outputs,
#but lyman wants them in the subject folder
for sub in subs:
    freesurfer_dir = home_dir + '/data/freesurfer/' + sub
    if os.path.exists(freesurfer_dir):
        sub_dir = home_dir + '/data/' + sub +'/'
        cmd = 'mv ' + freesurfer_dir + '/* ' + sub_dir
        os.system(cmd)
        os.rmdir(freesurfer_dir)